In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets , transforms

from torch.utils.data import DataLoader

transformss = transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor()])
trainset = datasets.FashionMNIST('MNIST_data/', download = True, train = True, transform = transformss)
testset = datasets.FashionMNIST('MNIST_data/', download = True, train = False, transform = transformss)
trainloader = DataLoader(trainset, batch_size = 600, shuffle = True)
testloader = DataLoader(testset, batch_size = 64, shuffle = True)

In [2]:
class Net(nn.Module):
    
    def  __init__(self):
        super().__init__()
        
        self.conv=nn.Sequential(
            nn.Conv2d(1,6,kernel_size=5,padding=0,stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(6),
            nn.MaxPool2d(2,stride=2), #dim=14*14
            
            nn.Conv2d(6,16,kernel_size=5,padding=0,stride=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,stride=2), #dim=6*6
            
            nn.Conv2d(16,32,kernel_size=3,padding=0,stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2)
            
        )
        self.linear=nn.Sequential(
            nn.Linear(32,10),
            nn.ReLU(),
            nn.Softmax(dim=-1)
        )
    
    def forward(self,image):
        
        output=self.conv(image)
        output=output.view(output.size(0),-1)
        output=self.linear(output)
        
        return output
    
    def ret_Image(self,image):
        
        output=self.conv(image)
        
        return output

In [3]:
def train(cnn,iterations,optimizer,loss_fn,trainloader):
    cnn.train()
    for i in range(iterations):
        
        lossperiter=0.0
        
        for j, data in enumerate(trainloader):
            
            img,label=data
            
            img=img.cuda()
            label=label.cuda()
            
            optimizer.zero_grad()
            output=cnn(img)
            loss=loss_fn(output,label)
            loss.backward()
            optimizer.step()
            lossperiter+=loss.item()
        
        print(lossperiter/600)

In [4]:
def test_network(testloader,cnn,optimizer,lossfunction):
    cnn.eval()
    total_correct = 0
    avg_loss = 0.0
    with  torch.no_grad():
        for i,data in enumerate(testloader):

            images,labels=data

            images=images.cuda()


            #calculate the output for each image
            output = cnn(images)
            #find the loss
            pred = output.detach().cpu().max(1)[1]
            total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(testset)
    print('Accuracy '+ str((float(total_correct)*100) / len(testset)))


In [5]:
cnn=Net()
iterations=10
optimizer=optim.SGD(cnn.parameters(),lr=0.08)
loss_fn=nn.CrossEntropyLoss()
cnn.cuda()
cnn

Net(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (9): ReLU()
    (10): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (linear): Sequential(
    (0): Linear(in_features=32, out_features=10, bias=True)
    (1): ReLU()
    (2): Softmax(dim=-1)
  )
)

In [6]:
train(cnn,iterations,optimizer,loss_fn,trainloader)

0.3709585110346476
0.3392715577284495
0.32131367246309916
0.3071698699394862
0.2979565093914668
0.2942353105545044
0.2921982479095459
0.29100979804992677
0.2898919222752253
0.28886417965094247


In [8]:
test_network(testloader,cnn,optimizer,loss_fn)

Accuracy 71.76


In [27]:
def test_heatmap(testloader,cnn):
    cnn.eval()
    counter=0
    for i,data in enumerate(testloader):

        images,_=data
        images=images.cuda()
        """img = cv2.imread('images', 1)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)"""
        output=cnn.ret_Image(images)
        output=output.detach().cpu().squeeze(dim=0)
        
        img=getUpsampled(output)
        
        while(counter<5):
            plt.imshow(img)
            counter+=1

    


In [28]:
def getUpsampled(output):
        
    m = Upsample(scale_factor=28, mode='bilinear')
    output_upsampled = m(output)
    output_upsampled = output_upsampled.view(output_upsampled.shape[1], output_upsampled.shape[2], output_upsampled.shape[0])
    
    return output_upsampled

In [29]:
from  torch.nn.modules.upsampling import Upsample

test_heatmap(testloader,cnn)
    

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


RuntimeError: shape '[32, 28, 64]' is invalid for input of size 1605632